In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../.credentials")

True

In [5]:
import pymongo
uri = os.environ.get("URI")
client = pymongo.MongoClient(uri)
client.list_database_names()

['sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

In [6]:
db = client.electronicsDB

In [7]:
# list the collections in the electronicsDB
db.collection_names()

/var/folders/26/wps0wjl102n8g8zxvtcvtx4w0000gn/T/ipykernel_6994/2955859118.py:2: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  db.collection_names()


[]

In [8]:
vg = db.video_games

In [9]:
# insert a new document with "title" and "year" fields - an "_id" will be assigned by MongoDB
# store the insert result in insert_result
insert_result = vg.insert_one({"title": "Fortnite", "year": 2018})

In [10]:
# check if the insert was acknowledged by the server
insert_result.acknowledged

True

In [11]:
# retrieve the "_id" value of the document we just inserted
insert_result.inserted_id

ObjectId('626e1b4606a6001b52bdbb67')

In [12]:
# find the document we just inserted by using its "_id"
# we can do this because "_id" is unique by collection
vg.find_one( { "_id": insert_result.inserted_id } )

{'_id': ObjectId('626e1b4606a6001b52bdbb67'),
 'title': 'Fortnite',
 'year': 2018}

## Upserts vs. Updates
Sometimes, we want to update a document, but we're not sure if it exists in the collection.

We can use an "upsert" to update a document if it exists, and insert it if it does not exist.

In the following example, we're not sure if this video game exists in our collection, but we want to make sure there is a document in the collection that contains the correct data.

This operation may do one of two things:

1. If the predicate matches a document, update the document to
   contain the correct data.

2. If the document doesn't exist, create the desired document.


In [13]:
fortnite_doc = {"title": "Fortnite", "year": 2018}

# the query predicate here is { "title": "Fortnite" }
# that will match the document we just inserted
# so this statement will update this document with the contents of "fortnite_doc"
upsert_result = vg.update_one( { "title": "Fortnite" } , { "$set": fortnite_doc }, upsert=True )

In [14]:
# this result object should have 'updatedExisting': True
# because this operation updated an existing document
upsert_result.raw_result

{'n': 1,
 'electionId': ObjectId('7fffffff0000000000000049'),
 'opTime': {'ts': Timestamp(1651383165, 2), 't': 73},
 'nModified': 0,
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1651383165, 2),
  'signature': {'hash': b'\xa3\x81Wt\x8f\x1d\x9d\x9d\xcdAG~\xd93\xcd|\xa7\xde\xddS',
   'keyId': 7051241787746156545}},
 'operationTime': Timestamp(1651383165, 2),
 'updatedExisting': True}

In [15]:
rocketleague_doc = {"title": "Rocket League", "year": 2015}

# the query predicate here is { "title": "Rocket League" }
# that won't match any documents, so this will insert "rocketleague_doc" as a new document
upsert_result = vg.update_one( { "title": "Rocket League" }, { "$set": rocketleague_doc }, upsert=True )

In [16]:
# this result object should have 'updatedExisting': False
# because this operation inserted a new document
# it also has an 'upserted' field with the "_id" of the new document
upsert_result.raw_result

{'n': 1,
 'electionId': ObjectId('7fffffff0000000000000049'),
 'opTime': {'ts': Timestamp(1651383196, 1082), 't': 73},
 'upserted': ObjectId('626e1b9ced406b03abbc4f69'),
 'nModified': 0,
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1651383196, 1082),
  'signature': {'hash': b"'vX\xd7\xf0\xf8\x04 \x0eN\x87\xdc\x86^\x1c\x1br\xb6^\xf8",
   'keyId': 7051241787746156545}},
 'operationTime': Timestamp(1651383196, 1082),
 'updatedExisting': False}